In [1]:
import igl

In [2]:
#https://github.com/libigl/libigl-tutorial-data

In [13]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from math import exp
import quaternion
import pickle

from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [170]:
V, F = igl.read_triangle_mesh('data/arm.obj')
C,BE,_,_,_,_ = igl.read_tgf('data/arm.tgf')
W = igl.read_dmat('data/arm-weights.dmat')
# labels = np.load('data/hand.label.npy').astype(int)
# v -= v.min(axis=0)
# v /= v.max()
mp.plot(V,F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

In [5]:
# TV = np.concatenate((V, C))
# TE = BE + V.shape[0]

In [6]:
# labels = np.concatenate((np.zeros(V.shape[0]), np.array(range(1,C.shape[0]+1)))).astype(int)
labels = np.array(range(BE.shape[0])).astype(int)

In [8]:
def circle_sum(q1, q2):
    return q1 + q2 if np.dot(quaternion.as_float_array(q1), quaternion.as_float_array(q2)) >= 0 else q1 - q2

def apply_weighted_rotation(vertices, wgts, quat, CoR, translation): #vertices, weights, rot (as quat), CoR, translation array 
    vnew = vertices.copy()
    for i in range(vertices.shape[0]):
        vi = vertices[i]
        wi = wgts[i]
        
        q = quaternion.as_quat_array([0,0,0,0])
        for j in range(BE.shape[0]): # #BE
            qj = quat[j]
            wij = wi[j]
            q = circle_sum(q, wij*qj)

        # wqi  = np.sum(qi, axis=0)
        q /= np.linalg.norm(quaternion.as_float_array(q))
        
        R = quaternion.as_rotation_matrix(q)
        
        Rtilda = np.zeros((3,3))
        ttilda = np.zeros((3,1))
        
        for j in range(BE.shape[0]):
            Rj = quaternion.as_rotation_matrix(quat[j]) #3 x 3
            tj = translation[j] # 1 x 3
            wij = wi[j]
            Rtilda += wij*Rj
            ttilda += wij*tj.reshape((3,1))
            
        t = Rtilda @ CoR[i].reshape((3,1)) + ttilda - R @ CoR[i].reshape((3,1))
        
        vnew[i] = (R @ vi.reshape((3,1)) + t).reshape(3)
            

        # vi_q = quaternion.as_quat_array(np.concatenate(([0], vi)))

        # qi_q = quaternion.as_quat_array(wqi)

        # res = qi_q.inverse() * vi_q * qi_q

        # vnew[i] = [res.x, res.y, res.z]

    return vnew

# def apply_weighted_rotation_linear(rot, vertices): 
#     vertices_new = np.zeros(vertices.shape)
#     for i in range(vertices.shape[0]):
#         vi = vertices[i,:]
#         wi = W[i,:]
        

       
#         # wqi /= np.linalg.norm(wqi)
        
#         r = Rotation.from_quat(rot)
        
#         rotmat = r.as_matrix() #4 by 3 x 3
        
#         for j in range(rot.shape[0]):
#             wj = wi[j]
#             Tj = rotmat[j,:]
            
#             qi = np.eye(Tj.shape[0]) * wj @ Tj
#             wqi  = qi @ vi
#             vertices_new[i] += wqi
        
#     return vertices_new

# def apply_rotation(rot, vertices):
#     vertices_new = vertices.copy()
#     for i in range(vertices.shape[0]):
#         vi = vertices[i,:]

#         vi_q = quaternion.as_quat_array(np.concatenate(([0], vi)))
#         qi_q = quaternion.as_quat_array(rot)
#         res = qi_q.inverse() * vi_q * qi_q

#         vertices_new[i] = [res.x, res.y, res.z]

#         # print(ri.shape)
#     return vertices_new

In [9]:
def similarity(Wp, Wv, sigma=0.1):
    #BE shape
    Wp = Wp if len(Wp.shape) == 1 else Wp.reshape(-1)
    Wv = Wv if len(Wv.shape) == 1 else Wv.reshape(-1)
    
    tot = 0
    for j in range(Wp.shape[0]):
        for k in range(j+1, Wv.shape[0]):
            tot += Wp[j]*Wp[k]*Wv[j]*Wv[k]*exp( -(Wp[j]*Wv[k] - Wp[k]*Wv[j])**2 / sigma**2)
            
    return tot

def CoR(i, weights, vertices, faces, omega=0.1, pb = None):
    
    num = np.zeros([1, 3])
    denom = np.zeros([1,3])
    for t in range(faces.shape[0]):
        # cmp = [i for i in range(3) if np.linalg.norm(weights[i] - weights[faces[t,i]]) < omega]
        cmp = [0, 1,2]
        if not cmp:
            continue
        s = similarity(weights[i], np.mean([weights[faces[t,c]] for c in cmp], axis = 0))
        v = np.mean([vertices[faces[t,c]] for c in cmp], axis = 0)
        a = igl.doublearea(vertices, faces[[t]]) / 2 

        num += s * v * a
        denom += s * a
    
    pi = num / denom
    
    if pb:
        pb.value = i

    return pi

In [21]:
P = np.zeros(V.shape)
with tqdm_joblib(tqdm(desc="My calculation", total=P.shape[0])) as progress_bar:
    P = np.array(Parallel(n_jobs=10)(delayed(CoR)(i, W, V, F) for i in range(P.shape[0])))

My calculation: 100%|███████████████████████| 8311/8311 [14:05<00:00,  9.83it/s]


In [22]:
mp.plot(P)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0007487…

In [11]:
# pickle.dump(P, open("data/CoR.p", "wb"))

In [140]:
P = pickle.load(open('data/CoR.p', 'rb'))

In [227]:
OCoR = V.copy()

pos_f_saver = np.zeros((labels.max()+1, 7))

def pos_f(s,x,y,z, w, α, β, γ):
    print(np.isin(labels, s))
    clicked_edges = BE[np.isin(labels, s)]
    slices = []
    for e in clicked_edges:
        for v in e:
            if v not in slices:
                slices.append(v)
                
    print(slices)
 
        
    # r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    r = Rotation.from_quat([α, β, γ, w])
    # r = np.array([w, α, β, γ])
    
    v_slice = C[slices] + np.array([[x,y,z]])
    
    # print(r.as_quat())
    
    # center = v_slice.mean(axis=0)
    C1 = C.copy()
    C1[slices] = r.apply(v_slice)
    # C1[slices] = apply_rotation(r, v_slice)
    for si in s:
        # print("si " ,si)
        pos_f_saver[si] = [x,y,z,w,α,β,γ]
    
    # Qt0  = quaternion.as_quat_array(igl.directed_edge_orientations(C,BE))
    # Qt1  =  quaternion.as_quat_array(igl.directed_edge_orientations(C1,BE))
    
    # qto * diff = qt1
    # diff = qt0^-1 * qt1
    # print(BE)
    
    q = np.zeros((BE.shape[0], 4))
    t = np.zeros((BE.shape[0], 3))
    for i in range(BE.shape[0]):
        # res =  Qt0[i].inverse() * Qt1[i] 
        # diff[i,:] = [res.w, res.x, res.y, res.z]
        xi,yi,zi,wi,αi,βi,γi = pos_f_saver[i]
        q[i,:] = [wi,αi,βi,γi]
        t[i,:] = [xi, yi, zi]
    
    # q /= np.linalg.norm(q, axis=1)
    print(q)
    Q = quaternion.as_quat_array(q)
    print(t)
    
    OCoR = V.copy()
    OCoR = apply_weighted_rotation(V, W, Q, P, t)
    

    v_deformed = pos_f.deformer(OCoR)
    p.update_object(vertices = v_deformed)
    p.remove_object(max(p._Viewer__objects.keys()))
    p.add_edges(C1+np.repeat([[0,.25,0]], C1.shape[0], axis=0), BE, shading={"line_color": "green"})
  
pos_f.deformer = lambda x:x

In [228]:
def widgets_wrapper():
    # segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    segment_widget = iw.SelectMultiple(options=np.arange(labels.max()+1))
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    # rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
    #                  for a in 'αβγ'}
    real_widget = {a:iw.FloatSlider(min=-2, max=2, value=1, step=.25) 
                     for a in 'w'}
    imag_widget = {a:iw.FloatSlider(min=-1, max=1, value=0, step=.1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value, real_widget['w'].value,
        imag_widget['α'].value,imag_widget['β'].value,
        imag_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(real_widget)
    widgets_dict.update(imag_widget)
    return widgets_dict

In [229]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [230]:

## Widget UI

p = mp.plot(V, F)
# p = mp.plot(v,f)
# p.add_points(VE)
p.add_edges(C,BE, shading={"line_color": "green"});
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

interactive(children=(SelectMultiple(description='s', options=(0, 1, 2, 3), value=()), FloatSlider(value=0.0, …

<function __main__.pos_f(s, x, y, z, w, α, β, γ)>

In [234]:
vQ

array([[ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 1.,  0.,  0., -2.]])

In [235]:
vT

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
###########

In [ ]:
# V, F = igl.read_triangle_mesh('data/cloth_ball0.ply')

In [ ]:
#get just the flat piece
# calculate the bbw (igl.bbw)

In [ ]:
# U,G = V,F

In [ ]:
_, U, G, _, _= igl.decimate(V, F, 5000)

In [ ]:
# plane_surf_F = G[igl.face_components(G) == 0]

In [ ]:
# plane_surf_V = U[list(set(plane_surf_F.reshape(-1))), :]

In [ ]:
# mp.plot(plane_surf_V, plane_surf_F, shading={"wireframe" : True})

In [ ]:
# lets first set constraints

In [ ]:
# BV,BF = igl.bounding_box(plane_surf_V, 0)

In [ ]:
# mp.plot(BV, BF)

In [ ]:
# ctrl_v = []
# mn = np.min(plane_surf_V,axis=0)
# # X = int((np.max(plane_surf_V,axis=0) - mn)[0])
# # Z = int((np.max(plane_surf_V,axis=0) - mn)[2])
# # for i in range(0, X+1, int(X /2)):
# #     for j in range (0, Z+1, int(Z/2)):
# #         ctrl_v.append(mn + [i, 0, j])
# for i in range(0, 60, 20):
#     for j in range (0, 60, 20):
#         ctrl_v.append(mn + [i, 0, j])
# ctrl_v = np.array(ctrl_v)

In [ ]:
# l = igl.boundary_loop(plane_surf_F)
# l = l[::11]
# lv = plane_surf_V[l,:]

In [ ]:
# # bbw_solver = igl.BBW()
# # bbw_solver.solve(plane_surf_V, plane_surf_F)
# ctr = np.median(V, axis=0)
# lv = np.concatenate((lv, [ctr]), axis=0)
# C = lv

In [ ]:
# ok = mp.plot(plane_surf_V,plane_surf_F)
# ok.add_points(C, shading={"point_size": 4.0})

In [ ]:
# P = np.array(range(C.shape[0]))

In [ ]:
# _, b, bc = igl.boundary_conditions(plane_surf_V,plane_surf_F, C, P , np.empty((0,2),dtype='int64'), np.empty((0,2),dtype='int64'))

In [ ]:
# bbw_solver = igl.BBW()
# bbw_solver.solve(plane_surf_V, plane_surf_F, b ,bc)

In [ ]:
# np.linalg.norm(plane_surf_V)

The skinning weights for the first four models were computed by bounded biharmonic weights with controlled extrema [Jacobson et al. 2012b] and the skinning weights for the cloth model were computed by Maya’s closest distance bind with a dropoff rate of 2.0

In [ ]:
# # wj(x0) = d(x0, Hj)^-1
# W = np.array([1 / np.linalg.norm(plane_surf_V - ctrl_v[i], axis=1) for i in range(ctrl_v.shape[0])])
# W = (W.T / np.linalg.norm(W, axis=0)[:,None])

In [ ]:
# W = (np.diag( 1 / np.linalg.norm(W,axis=0)) @ W.T)

In [ ]:
# mp.plot(plane_surf_V, plane_surf_F, c = W[:,4])

In [ ]:
# V = plane_surf_V
# F = plane_surf_F

# P = np.zeros(V.shape)

# with tqdm_joblib(tqdm(desc="My calculation", total=P.shape[0])) as progress_bar:
#     P = Parallel(n_jobs=10)(delayed(CoR)(i, W, V, F) for i in range(P.shape[0]))
# P = np.array(P)

In [ ]:
# pickle.dump(P, open('data/CoR_cloth_ball.p', 'wb'))

In [ ]:
# ok = mp.plot(V,F)
# ok.add_points(P, shading={"point_size": 1.0, 'coloring' : 'green'})